# Basic Attention Operation: Ungraded Lab

As you've learned, attention allows a seq2seq decoder to use information from each encoder step instead of just the final encoder hidden state. In the attention operation, the encoder outputs are weighted based on the decoder hidden state, then combined into one context vector. This vector is then used as input to the decoder to predict the next output step.

In this ungraded lab, you'll implement a basic attention operation as described in [Bhadanau, et al (2014)](https://arxiv.org/abs/1409.0473) using Numpy.

This is a practice notebook, where you can train writing your code. All of the solutions are provided at the end of the notebook.

In [1]:
# Import the libraries and define the functions you will need for this lab
import numpy as np

def softmax(x, axis=0):
    """ Calculate softmax function for an array x along specified axis
    
        axis=0 calculates softmax across rows which means each column sums to 1 
        axis=1 calculates softmax across columns which means each row sums to 1
    """
    return np.exp(x) / np.expand_dims(np.sum(np.exp(x), axis=axis), axis)

## 1: Calculating alignment scores

The first step is to calculate the alignment scores. This is a measure of similarity between the decoder hidden state and each encoder hidden state. From the paper, this operation looks like

$$
\large e_{ij} = v_a^\top \tanh{\left(W_a s_{i-1} + U_a h_j\right)}
$$

where $W_a \in \mathbb{R}^{n\times m}$, $U_a \in \mathbb{R}^{n \times m}$, and $v_a \in \mathbb{R}^m$
are the weight matrices and $n$ is the hidden state size. In practice, this is implemented as a feedforward neural network with two layers, where $m$ is the size of the layers in the alignment network. It looks something like:

![alignment model](./images/alignment_model_3.jpg)

Here $h_j$ are the encoder hidden states for each input step $j$ and $s_{i - 1}$ is the decoder hidden state of the previous step. The first layer corresponds to $W_a$ and $U_a$, while the second layer corresponds to $v_a$.

To implement this, first concatenate the encoder and decoder hidden states to produce an array with size $K \times 2n$ where $K$ is the number of encoder states/steps. For this, use `np.concatenate` ([docs](https://numpy.org/doc/stable/reference/generated/numpy.concatenate.html)). Note that there is only one decoder state so you'll need to reshape it to successfully concatenate the arrays. The easiest way is to use `decoder_state.repeat` ([docs](https://numpy.org/doc/stable/reference/generated/numpy.repeat.html#numpy.repeat)) to match the hidden state array size.

Then, apply the first layer as a matrix multiplication between the weights and the concatenated input. Use the tanh function to get the activations. Finally, compute the matrix multiplication of the second layer weights and the activations. This returns the alignment scores.

In [2]:
hidden_size = 16
attention_size = 10
input_length = 5

np.random.seed(42)

# Synthetic vectors used to test
encoder_states = np.random.randn(input_length, hidden_size)
decoder_state = np.random.randn(1, hidden_size)

# Weights for the neural network, these are typically learned through training
# Use these in the alignment function below as the layer weights
layer_1 = np.random.randn(2 * hidden_size, attention_size)
layer_2 = np.random.randn(attention_size, 1)

# Implement this function. Replace None with your code. Solution at the bottom of the notebook
def alignment(encoder_states, decoder_state):
    # First, concatenate the encoder states and the decoder state
    inputs = np.concatenate((encoder_states, decoder_state.repeat(input_length, axis=0)), axis=1)
    assert inputs.shape == (input_length, 2 * hidden_size)
    
    # Matrix multiplication of the concatenated inputs and layer_1, with tanh activation
    activations = np.tanh(np.matmul(inputs, layer_1))
    assert activations.shape == (input_length, attention_size)
    
    # Matrix multiplication of the activations with layer_2. Remember that you don't need tanh here
    scores = np.matmul(activations, layer_2)
    assert scores.shape == (input_length, 1)
    
    return scores

In [3]:
# Print the encoder states
print(f"Encoder states looks like this: \n{encoder_states[:1]}\n\n")

# Print the decoder state
print(f"Decoder state looks like this: \n{decoder_state[:1]}")

Encoder states looks like this: 
[[ 0.49671415 -0.1382643   0.64768854  1.52302986 -0.23415337 -0.23413696
   1.57921282  0.76743473 -0.46947439  0.54256004 -0.46341769 -0.46572975
   0.24196227 -1.91328024 -1.72491783 -0.56228753]]


Decoder state looks like this: 
[[-0.21967189  0.35711257  1.47789404 -0.51827022 -0.8084936  -0.50175704
   0.91540212  0.32875111 -0.5297602   0.51326743  0.09707755  0.96864499
  -0.70205309 -0.32766215 -0.39210815 -1.46351495]]


In [4]:
# First, concatenate the encoder states and decoder state
inputs = np.concatenate((encoder_states, decoder_state.repeat(input_length, axis=0)), axis=1)
assert inputs.shape == (input_length, 2 * hidden_size)

print(f"Inputs looks like this \n\n{inputs[:1]}")

Inputs looks like this 

[[ 0.49671415 -0.1382643   0.64768854  1.52302986 -0.23415337 -0.23413696
   1.57921282  0.76743473 -0.46947439  0.54256004 -0.46341769 -0.46572975
   0.24196227 -1.91328024 -1.72491783 -0.56228753 -0.21967189  0.35711257
   1.47789404 -0.51827022 -0.8084936  -0.50175704  0.91540212  0.32875111
  -0.5297602   0.51326743  0.09707755  0.96864499 -0.70205309 -0.32766215
  -0.39210815 -1.46351495]]


In [5]:
# Print the layer 1
print(f"Layer 1 looks like this: \n{layer_1[:1]}\n\n")

# Print the layer 2
print(f"Layer 2 looks like this: \n{layer_2[:1]}")

Layer 1 looks like this: 
[[ 0.29612028  0.26105527  0.00511346 -0.23458713 -1.41537074 -0.42064532
  -0.34271452 -0.80227727 -0.16128571  0.40405086]]


Layer 2 looks like this: 
[[1.55115198]]


In [ ]:
# Matrix multiplication of the concatenated inputs and layer_1, with tanh activation
activations = np.tanh(np.matmul(inputs, layer_1))
assert activations.shape == (input_length, attention_size)

print(f"Activations looks like this: \n\n{activations[:1]}")

In [7]:
# Matrix multiplication of the activations with layer_2. Remember that you don't need tanh here
scores = np.matmul(activations, layer_2)
assert scores.shape == (input_length, 1)

print(f"Scores looks like this: \n\n{scores[:1]}")

Scores looks like this: 

[[4.35790943]]


In [8]:
# Run this to test your alignment function
scores = alignment(encoder_states, decoder_state)
print(scores)

[[4.35790943]
 [5.92373433]
 [4.18673175]
 [2.11437202]
 [0.95767155]]


If you implemented the function correctly, you should get these scores:

```python
[[4.35790943]
 [5.92373433]
 [4.18673175]
 [2.11437202]
 [0.95767155]]
```

## 2: Turning alignment into weights

The next step is to calculate the weights from the alignment scores. These weights determine the encoder outputs that are the most important for the decoder output. These weights should be between 0 and 1. You can use the softmax function (which is already implemented above) to get these weights from the attention scores. Pass the attention scores vector to the softmax function to get the weights. Mathematically,

$$
\large \alpha_{ij} = \frac{\exp{\left(e_{ij}\right)}}{\sum_{k=1}^K \exp{\left(e_{ik}\right)}}
$$



## 3: Weight the encoder output vectors and sum

The weights tell you the importance of each input word with respect to the decoder state. In this step, you use the weights to modulate the magnitude of the encoder vectors. Words with little importance will be scaled down relative to important words. Multiply each encoder vector by its respective weight to get the alignment vectors, then sum up the weighted alignment vectors to get the context vector. Mathematically,

$$
\large c_i = \sum_{j=1}^K\alpha_{ij} h_{j}
$$

Implement these steps in the `attention` function below.

In [9]:
hidden_size = 16
attention_size = 10
input_length = 5

np.random.seed(42)

# Synthetic vectors used to test
encoder_states = np.random.randn(input_length, hidden_size)
decoder_state = np.random.randn(1, hidden_size)

In [10]:
print(f"Encoder states: \n{encoder_states}\n\n")
print(f"Decoder state: \n{decoder_state}")

Encoder states: 
[[ 0.49671415 -0.1382643   0.64768854  1.52302986 -0.23415337 -0.23413696
   1.57921282  0.76743473 -0.46947439  0.54256004 -0.46341769 -0.46572975
   0.24196227 -1.91328024 -1.72491783 -0.56228753]
 [-1.01283112  0.31424733 -0.90802408 -1.4123037   1.46564877 -0.2257763
   0.0675282  -1.42474819 -0.54438272  0.11092259 -1.15099358  0.37569802
  -0.60063869 -0.29169375 -0.60170661  1.85227818]
 [-0.01349722 -1.05771093  0.82254491 -1.22084365  0.2088636  -1.95967012
  -1.32818605  0.19686124  0.73846658  0.17136828 -0.11564828 -0.3011037
  -1.47852199 -0.71984421 -0.46063877  1.05712223]
 [ 0.34361829 -1.76304016  0.32408397 -0.38508228 -0.676922    0.61167629
   1.03099952  0.93128012 -0.83921752 -0.30921238  0.33126343  0.97554513
  -0.47917424 -0.18565898 -1.10633497 -1.19620662]
 [ 0.81252582  1.35624003 -0.07201012  1.0035329   0.36163603 -0.64511975
   0.36139561  1.53803657 -0.03582604  1.56464366 -2.6197451   0.8219025
   0.08704707 -0.29900735  0.09176078 -1.9

In [11]:
# First, calculate the alignment scores
scores = alignment(encoder_states, decoder_state)

print(scores)
    

[[4.35790943]
 [5.92373433]
 [4.18673175]
 [2.11437202]
 [0.95767155]]


In [12]:
# Then take the softmax of the alignment scores to get a weight distribution
weights = softmax(scores)

print(weights)

[[0.14773795]
 [0.70716569]
 [0.12449461]
 [0.01567242]
 [0.00492933]]


In [13]:
# Multiply each encoder state by its respective weight
weighted_scores = encoder_states * weights

print(weighted_scores)

[[ 7.33835307e-02 -2.04268844e-02  9.56881768e-02  2.25009309e-01
  -3.45933396e-02 -3.45909140e-02  2.33309664e-01  1.13379234e-01
  -6.93591833e-02  8.01567086e-02 -6.84643799e-02 -6.88059590e-02
   3.57470100e-02 -2.82664101e-01 -2.54835824e-01 -8.30712069e-02]
 [-7.16239420e-01  2.22224932e-01 -6.42123473e-01 -9.98732724e-01
   1.03645653e+00 -1.59661254e-01  4.77536296e-02 -1.00753304e+00
  -3.84968786e-01  7.84406499e-02 -8.13943169e-01  2.65680749e-01
  -4.24751075e-01 -2.06275812e-01 -4.25506273e-01  1.30986758e+00]
 [-1.68033172e-03 -1.31679309e-01  1.02402408e-01 -1.51988453e-01
   2.60023917e-02 -2.43968367e-01 -1.65352003e-01  2.45081627e-02
   9.19351084e-02  2.13344272e-02 -1.43975877e-02 -3.74857870e-02
  -1.84068018e-01 -8.96167235e-02 -5.73470438e-02  1.31606019e-01]
 [ 5.38533124e-03 -2.76311114e-02  5.07918111e-03 -6.03517245e-03
  -1.06090080e-02  9.58644964e-03  1.61582608e-02  1.45954161e-02
  -1.31525722e-02 -4.84610720e-03  5.19170068e-03  1.52891560e-02
  -7.50

In [14]:
# Sum up 
context = np.sum(weighted_scores, axis=0)

print(context)

[-0.63514569  0.04917298 -0.43930867 -0.9268003   1.01903919 -0.43181409
  0.13365099 -0.84746874 -0.37572203  0.18279832 -0.90452701  0.17872958
 -0.58015282 -0.58294027 -0.75457577  1.32985756]


In [16]:
# Implement this function. Replace None with your code.
def attention(encoder_states, decoder_state):
    """ Example function that calculates attention, returns the context vector 
    
        Arguments:
        encoder_vectors: NxM numpy array, where N is the number of vectors and M is the vector length
        decoder_vector: 1xM numpy array, M is the vector length, much be the same M as encoder_vectors
    """ 
    
    # First, calculate the alignment scores
    scores = alignment(encoder_states, decoder_state)
    
    # Then take the softmax of the alignment scores to get a weight distribution
    weights = softmax(scores)
    
    # Multiply each encoder state by its respective weight
    weighted_scores = encoder_states * weights
    
    # Sum up weighted alignment vectors to get the context vector and return it
    context = np.sum(weighted_scores, axis=0)
    return context

context_vector = attention(encoder_states, decoder_state)
print(context_vector)

[-0.63514569  0.04917298 -0.43930867 -0.9268003   1.01903919 -0.43181409
  0.13365099 -0.84746874 -0.37572203  0.18279832 -0.90452701  0.17872958
 -0.58015282 -0.58294027 -0.75457577  1.32985756]


If you implemented the `attention` function correctly, the context vector should be

```python
[-0.63514569  0.04917298 -0.43930867 -0.9268003   1.01903919 -0.43181409
  0.13365099 -0.84746874 -0.37572203  0.18279832 -0.90452701  0.17872958
 -0.58015282 -0.58294027 -0.75457577  1.32985756]
```



## See below for solutions

```python
# Solution
def alignment(encoder_states, decoder_state):
    # First, concatenate the encoder states and the decoder state.
    inputs = np.concatenate((encoder_states, decoder_state.repeat(input_length, axis=0)), axis=1)
    assert inputs.shape == (input_length, 2*hidden_size)
    
    # Matrix multiplication of the concatenated inputs and the first layer, with tanh activation
    activations = np.tanh(np.matmul(inputs, layer_1))
    assert activations.shape == (input_length, attention_size)
    
    # Matrix multiplication of the activations with the second layer. Remember that you don't need tanh here
    scores = np.matmul(activations, layer_2)
    assert scores.shape == (input_length, 1)
    
    return scores

# Run this to test your alignment function
scores = alignment(encoder_states, decoder_state)
print(scores)
```

```python
# Solution
def attention(encoder_states, decoder_state):
    """ Example function that calculates attention, returns the context vector 
    
        Arguments:
        encoder_vectors: NxM numpy array, where N is the number of vectors and M is the vector length
        decoder_vector: 1xM numpy array, M is the vector length, much be the same M as encoder_vectors
    """ 
    
    # First, calculate the dot product of each encoder vector with the decoder vector
    scores = alignment(encoder_states, decoder_state)
    
    # Then take the softmax of those scores to get a weight distribution
    weights = softmax(scores)
    
    # Multiply each encoder state by its respective weight
    weighted_scores = encoder_states * weights
    
    # Sum up the weights encoder states
    context = np.sum(weighted_scores, axis=0)
    
    return context

context_vector = attention(encoder_states, decoder_state)
print(context_vector)
```